In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [2]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT'):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0    nameFileS:parameters.json
1    nameFileS:parameters.json
2    nameFileS:parameters.json
3    nameFileS:parameters.json
4    nameFileS:parameters.json
5    nameFileS:parameters.json
6    nameFileS:parameters.json
7    nameFileS:parameters.json
8    nameFileS:parameters.json
9    nameFileS:parameters.json
dtype: object
0    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/binar...
1    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/floyd...
2    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/bfs/p...
3    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/banke...
4    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/bellm...
5    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT/selec...
6    pathFileS:../../../SLIDE-x-D003-Arm-Thumb

In [3]:
dfRDC = pd.read_csv("RangeValueClassification_tot.csv", sep=';')
del dfRDC['Unnamed: 0']

In [4]:
dfRDC

,DEVICE,FUNCTION,DATA_TYPE,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,Arm,binary_search,int8_t,1,2,1,3,1,1,0,0
1,Arm,binary_search,int16_t,1,2,1,3,1,1,0,0
2,Arm,binary_search,int32_t,1,2,1,3,1,1,0,0
3,Arm,binary_search,int64_t,1,2,1,3,1,1,0,0
4,Thumb,binary_search,int8_t,1,2,1,3,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
195,RiscV,gcd,int64_t,2,2,0,0,0,0,0,0
196,Leon3,gcd,int8_t,2,2,0,0,0,0,0,0
197,Leon3,gcd,int16_t,2,2,0,0,0,0,0,0
198,Leon3,gcd,int32_t,2,2,0,0,0,0,0,0


In [5]:
nameFileH = [] # Affinity
pathFileH = [] # Affinity
rootFileH = [] # Affinity
nameFileM = [] # cc4csValues
pathFileM = [] # cc4csValues 
rootFileM = [] # cc4csValues
nameFileS = [] # Size SW
pathFileS = [] # Size SW
rootFileS = [] # Size SW
#nameFileC = []
#pathFileC = []
#rootFileC = []
nameFileA = [] # area HW
pathFileA = [] # area HW
rootFileA = [] # area HW

# path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_INT'):
    for file in files:
        if file.endswith("Affinity.csv"):
            nameFileH.append(file)
            pathFileH.append(os.path.join(root, file))
            rootFileH.append(root)
        elif file.endswith("cc4csValues.csv"):
            nameFileM.append(file)
            pathFileM.append(os.path.join(root, file))
            rootFileM.append(root)
        elif file.endswith("Size.csv"):
            nameFileS.append(file)
            pathFileS.append(os.path.join(root, file))
            rootFileS.append(root)
        elif file.endswith("idAreaValuesParam.csv"):
            nameFileA.append(file)
            pathFileA.append(os.path.join(root, file))
            rootFileA.append(root)

#print('nameFileH'+str(nameFileH))
#print('pathFileH'+str(pathFileH))
#print('rootFileH'+str(rootFileH))

#print('nameFileM'+str(nameFileM))
#print('pathFileM'+str(pathFileM))
#print('rootFileM'+str(rootFileM))

#print('nameFileS'+str(nameFileH))
#print('pathFileS'+str(nameFileH))
#print('rootFileS'+str(nameFileH))

#print('nameFileA'+str(nameFileH))
#print('pathFileA'+str(nameFileH))
#print('rootFileA'+str(nameFileH))

In [6]:
# Affinity file
for i in range(0, len(pathFileH)):
    path = pathFileH[i]
    try:
        dfHTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)
        
    if os.name == 'nt':
        spR = rootFileH[i].split('\\')
    else:
        spR = rootFileH[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfHTemp['DEVICE'] = device
    dfHTemp['BOARD'] = board
    dfHTemp['FUNCTION'] = function
    dfHTemp['DATA_TYPE'] = datatype
    dfHTemp['OPT_FLAG'] = optFlag
    dfHTemp['MV_SIZE'] = matrixVectorSize
    dfHTemp['inputName'] = dfHTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfHTemp['inputName'] = dfHTemp['inputName'].astype(int)
    dfHTemp.sort_values('inputName')
    
    if i == 0:
        dfHTot = dfHTemp
    else:
        frames = [dfHTot, dfHTemp]
        dfHTot = pd.concat(frames)
        
# a = dfHTot['Program_Volume']/3000
# a = a.values.tolist()
# dfHTot['Bugs_Delivered'] = a

In [7]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,185,35,74,30,259,65,1559.79,67331.08,0.02,...,0.021622,0.418919,0.862829,0.663393,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
1,19,187,35,74,30,261,65,1571.84,67851.01,0.02,...,0.021390,0.418919,0.862829,0.663146,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
2,12,183,35,74,30,257,65,1547.75,66811.14,0.02,...,0.021858,0.418919,0.862829,0.663644,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
3,38,209,35,89,45,298,80,1883.93,65205.07,0.03,...,0.019139,0.348315,0.832252,0.574210,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
4,81,179,35,72,28,251,63,1500.30,67513.38,0.02,...,0.022346,0.430556,0.867085,0.676244,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
96,98,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
97,29,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
98,80,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32


In [8]:
# Define the function to categorize clock cycles
def categorize_opt(clockCycles, minCC, q1CC, medCC, q3CC, maxCC):
    if clockCycles == minCC:
        return 1
    elif  clockCycles > minCC and clockCycles < q1CC:
        return 2
    elif clockCycles >= q1CC and clockCycles < medCC:
        return 3
    elif clockCycles >= medCC and clockCycles < q3CC:
        return 4
    elif clockCycles >= q3CC and clockCycles < maxCC:
        return 5
    elif clockCycles == maxCC:
        return 6
    
def has_number(column):
    # Use apply to check each value and any to see if any are numeric
    return column.apply(lambda x: isinstance(x, (int, float))).any()

# cc4csValues file
for i in range(0, len(pathFileM)):
    path = pathFileM[i]
    dfMTemp = pd.read_csv(path, sep=',', quotechar='"',engine='python')
        
    if os.name == 'nt':
        spR = rootFileM[i].split('\\')
    else:
        spR = rootFileM[i].split('/')  
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    board = spR[-4] 
    function = spR[-5]
    dfMTemp['DEVICE'] = device
    dfMTemp['BOARD'] = board
    dfMTemp['FUNCTION'] = function
    dfMTemp['DATA_TYPE'] = datatype
    dfMTemp['OPT_FLAG'] = optFlag
    dfMTemp['MV_SIZE'] = matrixVectorSize
    dfMTemp['inputName'] = dfMTemp['id'].map(lambda x: x.lstrip('values_'))
    dfMTemp['inputName'] = dfMTemp['inputName'].astype(int)
    dfMTemp.sort_values('inputName')
    dfMTemp.drop('id', axis=1, inplace=True)
    
    if (not has_number(dfMTemp['clockCycles'])):
        dfMTemp['clockCycles'] = 0
    
    if i == 0:
        dfMTot = dfMTemp
    else:
        frames = [dfMTot, dfMTemp]
        dfMTot = pd.concat(frames)

In [9]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,4200.0,1441.0,84.8,86.1,53.1,17.15,17.15,0.0,79,201,...,14.737,14.189,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,11,NaN
1,4200.0,1441.0,84.8,86.1,53.1,17.15,17.15,0.0,79,201,...,14.737,14.189,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,19,NaN
2,4181.0,1428.0,84.6,85.9,54.0,17.08,17.08,0.0,68,169,...,16.396,15.777,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,12,NaN
3,4451.0,1611.0,86.1,87.4,52.3,18.10,18.10,0.0,138,335,...,7.850,7.609,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,38,NaN
4,4166.0,1421.0,84.6,85.9,54.0,17.05,17.05,0.0,66,161,...,17.728,17.144,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,990.0,990.0,NaN,NaN,NaN,NaN,NaN,NaN,93,170,...,4.231,3.640,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,3,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571722', '0..."
96,973.0,973.0,NaN,NaN,NaN,NaN,NaN,NaN,89,169,...,4.889,4.158,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,98,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571654', '0..."
97,952.0,952.0,NaN,NaN,NaN,NaN,NaN,NaN,65,118,...,5.950,5.118,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,29,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571606', '0..."
98,955.0,955.0,NaN,NaN,NaN,NaN,NaN,NaN,71,132,...,5.788,4.948,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,80,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571608', '0..."


In [10]:
# Size SW file
for i in range(0, len(pathFileS)):
    path = pathFileS[i]
    dfSTemp = pd.read_csv(path, sep=',',engine='python')

    if os.name == 'nt':
        spR = rootFileS[i].split('\\')
    else:
        spR = rootFileS[i].split('/')   
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = spR[-4]
    function = spR[-5]
    dfSTemp['DEVICE'] = device
    dfSTemp['BOARD'] = board
    dfSTemp['FUNCTION'] = function
    dfSTemp['DATA_TYPE'] = datatype
    dfSTemp['OPT_FLAG'] = optFlag
    dfSTemp['MV_SIZE'] = matrixVectorSize
    dfSTemp['inputName'] = dfSTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(int)
    dfSTemp.sort_values('inputName')
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].map(lambda x: x.lstrip('values_'))
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].astype(int)
    #dfSTemp.sort_values(by=dfSTemp.columns[0])
    
    if i == 0:
        dfSTot = dfSTemp
    else:
        frames = [dfSTot, dfSTemp]
        dfSTot = pd.concat(frames)

In [11]:
# dfSTemp.drop(dfSTemp.index, inplace=True)
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
1,19,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
2,12,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
3,38,11840.0,192.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
4,81,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
95,3,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
96,98,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
97,29,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
98,80,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32


In [12]:
# area parameters file
for i in range(0, len(pathFileA)):
    path = pathFileA[i]
    dfATemp = pd.read_csv(path, sep=',',engine='python')
    
    if os.name == 'nt':
        spR = rootFileA[i].split('\\')
    else:
        spR = rootFileA[i].split('/')
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    function = spR[-5]
    dfATemp['DEVICE'] = device
    dfATemp['BOARD'] = board
    dfATemp['FUNCTION'] = function
    dfATemp['DATA_TYPE'] = datatype
    dfATemp['OPT_FLAG'] = optFlag
    dfATemp['MV_SIZE'] = matrixVectorSize
    dfATemp['inputName'] = dfATemp['ID_VAL'].map(lambda x: x.lstrip('values_'))
    dfATemp['inputName'] = dfATemp['inputName'].astype(int)
    dfATemp.sort_values('ID_VAL')
    
    if i == 0:
        dfATot = dfATemp
    else:
        frames = [dfATot, dfATemp]
        dfATot = pd.concat(frames)

b = dfATot['Clock_Cycles']/dfATot['Max_Freq']
b = b.values.tolist()
dfATot['Time'] = b

c = (30/dfATot['Max_Freq'])*dfATot['Clock_Cycles']
c = c.values.tolist()
dfATot['Energy'] = c

NameError: name 'dfATot' is not defined

In [13]:
dfATot

NameError: name 'dfATot' is not defined

In [14]:
dfHTot = dfHTot.reset_index(drop=True)
dfMTot = dfMTot.reset_index(drop=True)
dfSTot = dfSTot.reset_index(drop=True)
#dfCTot = dfCTot.reset_index(drop=True)
# dfATot = dfATot.reset_index(drop=True)

In [15]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,185,35,74,30,259,65,1559.79,67331.08,0.02,...,0.021622,0.418919,0.862829,0.663393,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
1,19,187,35,74,30,261,65,1571.84,67851.01,0.02,...,0.021390,0.418919,0.862829,0.663146,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
2,12,183,35,74,30,257,65,1547.75,66811.14,0.02,...,0.021858,0.418919,0.862829,0.663644,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
3,38,209,35,89,45,298,80,1883.93,65205.07,0.03,...,0.019139,0.348315,0.832252,0.574210,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
4,81,179,35,72,28,251,63,1500.30,67513.38,0.02,...,0.022346,0.430556,0.867085,0.676244,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435295,3,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435296,98,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435297,29,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435298,80,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.574468,0.928455,0.774615,RiscV,RiscV,gcd,int8_t,optO2-07,32x32


In [16]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,4200.0,1441.0,84.8,86.1,53.1,17.15,17.15,0.0,79,201,...,14.737,14.189,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,11,NaN
1,4200.0,1441.0,84.8,86.1,53.1,17.15,17.15,0.0,79,201,...,14.737,14.189,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,19,NaN
2,4181.0,1428.0,84.6,85.9,54.0,17.08,17.08,0.0,68,169,...,16.396,15.777,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,12,NaN
3,4451.0,1611.0,86.1,87.4,52.3,18.10,18.10,0.0,138,335,...,7.850,7.609,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,38,NaN
4,4166.0,1421.0,84.6,85.9,54.0,17.05,17.05,0.0,66,161,...,17.728,17.144,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435295,990.0,990.0,NaN,NaN,NaN,NaN,NaN,NaN,93,170,...,4.231,3.640,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,3,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571722', '0..."
435296,973.0,973.0,NaN,NaN,NaN,NaN,NaN,NaN,89,169,...,4.889,4.158,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,98,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571654', '0..."
435297,952.0,952.0,NaN,NaN,NaN,NaN,NaN,NaN,65,118,...,5.950,5.118,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,29,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571606', '0..."
435298,955.0,955.0,NaN,NaN,NaN,NaN,NaN,NaN,71,132,...,5.788,4.948,RiscV,RiscV,gcd,int8_t,optO2-07,32x32,80,"['960555', '8845078', '126596', '1107131', '65034', '552257', '568396', '50.035', '12571608', '0..."


In [17]:
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
1,19,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
2,12,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
3,38,11840.0,192.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
4,81,11840.0,160.0,368.0,Leon3,Leon3,binary_search,int16_t,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
435295,3,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435296,98,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435297,29,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32
435298,80,5832.0,2500.0,1472.0,RiscV,RiscV,gcd,int8_t,optO2-07,32x32


In [18]:
dfATot

NameError: name 'dfATot' is not defined

In [19]:
# To get the common columns between the DataFrames, we can use set intersection
common_columns = set(dfHTot.columns).intersection(set(dfMTot.columns)).intersection(set(dfSTot.columns))

# Convert the intersection to a list
common_columns = list(common_columns)

# Merge the DataFrames using the common columns
dfTot = pd.merge(dfHTot, dfMTot, on=common_columns, how='outer')
dfTot = pd.merge(dfTot, dfSTot, on=common_columns, how='outer') # SW
# dfTot = pd.merge(dfTot, dfATot, on=common_columns, how='outer') # HW

# Display the resulting DataFrame
# print(dfTot)

In [20]:
dfTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,2016,66,1296,1036,3312,1102,33470.77,1381735.37,0.02,...,83.33,60.0,3.841,0.655,1.364,1.814,NaN,972.0,1800.0,132.0
1,0,2016,66,1296,1036,3312,1102,33470.77,1381735.37,0.02,...,83.33,60.0,3.670,0.626,1.303,1.718,NaN,948.0,1800.0,132.0
2,0,2016,66,1296,1036,3312,1102,33470.77,1381735.37,0.02,...,83.33,60.0,4.179,0.712,1.484,1.974,NaN,992.0,1800.0,132.0
3,0,2016,66,1296,1036,3312,1102,33470.77,1381735.37,0.02,...,83.33,60.0,3.709,0.632,1.317,1.578,NaN,952.0,1800.0,132.0
4,0,2016,66,1296,1036,3312,1102,33470.77,1381735.37,0.02,...,83.33,60.0,3.648,0.622,1.299,1.732,NaN,1072.0,1800.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443248,99,178,27,73,35,251,62,1494.50,42080.94,0.04,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0
443249,99,178,27,73,35,251,62,1494.50,42080.94,0.04,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0
443250,99,178,27,73,35,251,62,1494.50,42080.94,0.04,...,100.00,100.0,6.098,1.160,1.096,2.024,NaN,80.0,23.0,0.0
443251,99,178,27,73,35,251,62,1494.50,42080.94,0.04,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0


In [21]:
dfTot = dfTot[ ['inputName','DEVICE','BOARD','FUNCTION','DATA_TYPE','OPT_FLAG','MV_SIZE'] + [ col for col in dfTot.columns if (col != 'inputName' and col != 'DEVICE' and col != 'BOARD' and col != 'FUNCTION' and col != 'DATA_TYPE' and col != 'OPT_FLAG' and col != 'MV_SIZE') ] ]

In [22]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,Arm,Arm,astar,int16_t,optO1-00,32x32,2016,66,1296,...,83.33,60.0,3.841,0.655,1.364,1.814,NaN,972.0,1800.0,132.0
1,0,Arm,Arm,astar,int16_t,optO1-01,32x32,2016,66,1296,...,83.33,60.0,3.670,0.626,1.303,1.718,NaN,948.0,1800.0,132.0
2,0,Arm,Arm,astar,int16_t,optO1-02,32x32,2016,66,1296,...,83.33,60.0,4.179,0.712,1.484,1.974,NaN,992.0,1800.0,132.0
3,0,Arm,Arm,astar,int16_t,optO1-03,32x32,2016,66,1296,...,83.33,60.0,3.709,0.632,1.317,1.578,NaN,952.0,1800.0,132.0
4,0,Arm,Arm,astar,int16_t,optO2-00,32x32,2016,66,1296,...,83.33,60.0,3.648,0.622,1.299,1.732,NaN,1072.0,1800.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443248,99,Thumb,Thumb,selectionsort,int8_t,optO3-04,32x32,178,27,73,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0
443249,99,Thumb,Thumb,selectionsort,int8_t,optO3-05,32x32,178,27,73,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0
443250,99,Thumb,Thumb,selectionsort,int8_t,optO4-Os,32x32,178,27,73,...,100.00,100.0,6.098,1.160,1.096,2.024,NaN,80.0,23.0,0.0
443251,99,Thumb,Thumb,selectionsort,int8_t,optO5-Ofast,32x32,178,27,73,...,100.00,100.0,5.558,1.057,0.999,1.658,NaN,104.0,23.0,0.0


In [23]:
dfTot = pd.merge(dfTot,dfRDC,how='left',on=['DEVICE','FUNCTION','DATA_TYPE'])

In [24]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,data,bss,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,0,Arm,Arm,astar,int16_t,optO1-00,32x32,2016,66,1296,...,1800.0,132.0,1,2,1,3,0,0,1,1
1,0,Arm,Arm,astar,int16_t,optO1-01,32x32,2016,66,1296,...,1800.0,132.0,1,2,1,3,0,0,1,1
2,0,Arm,Arm,astar,int16_t,optO1-02,32x32,2016,66,1296,...,1800.0,132.0,1,2,1,3,0,0,1,1
3,0,Arm,Arm,astar,int16_t,optO1-03,32x32,2016,66,1296,...,1800.0,132.0,1,2,1,3,0,0,1,1
4,0,Arm,Arm,astar,int16_t,optO2-00,32x32,2016,66,1296,...,1800.0,132.0,1,2,1,3,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443248,99,Thumb,Thumb,selectionsort,int8_t,optO3-04,32x32,178,27,73,...,23.0,0.0,0,0,1,3,1,1,0,0
443249,99,Thumb,Thumb,selectionsort,int8_t,optO3-05,32x32,178,27,73,...,23.0,0.0,0,0,1,3,1,1,0,0
443250,99,Thumb,Thumb,selectionsort,int8_t,optO4-Os,32x32,178,27,73,...,23.0,0.0,0,0,1,3,1,1,0,0
443251,99,Thumb,Thumb,selectionsort,int8_t,optO5-Ofast,32x32,178,27,73,...,23.0,0.0,0,0,1,3,1,1,0,0


In [25]:
dfTot.rename(columns={'NUM_S':'SCALAR_INPUT',
                      'RANGE_S':'RANGE_SCALAR_VALUES',
                      'NUM_SI':'SCALAR_INDEX_INPUT',
                      'RANGE_SI':'RANGE_SCALAR_INDEX_VALUES',
                      'NUM_ARR':'ARRAY_INPUT',
                      'RANGE_ARR':'RANGE_ARRAY_INPUT',
                      'NUM_MAT':'MAT_INPUT',
                      'RANGE_MAT':'RANGE_MAT_INPUT'}, inplace=True)

In [26]:
dfTot.to_csv("TotalParameterMatrix.csv", sep=';', index=False, encoding='utf-8')

In [27]:
import pandas as pd
import os

def split_csv_by_column(input_file, column_name):
    # Read the CSV file
    df = pd.read_csv(input_file, sep=';')

    # Get unique values in the specified column
    unique_values = df[column_name].unique()

    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)

    # For each unique value, create a subset of the DataFrame and save it as a CSV file
    for value in unique_values:
        subset_df = df[df[column_name] == value]
        output_file = os.path.join(input_dir, f"{value}-KERNEL-INT.csv")
        subset_df.to_csv(output_file, index=False)
        print(f"Created file: {output_file}")

input_file = 'Arm-Thumb-Atmega-Leon3-RiscV-KERNEL-INT.csv'  # replace with the path to your CSV file
column_name = 'DEVICE'  # replace with the name of the column to split by

split_csv_by_column(input_file, column_name)

Created file: Arm-KERNEL-INT.csv
Created file: Atmega328p-KERNEL-INT.csv
Created file: Leon3-KERNEL-INT.csv
Created file: RiscV-KERNEL-INT.csv
Created file: Thumb-KERNEL-INT.csv


In [21]:
import pandas as pd
import os

def merge_csv_files(file1, file2):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Concatenate the DataFrames, axis=1 to concatenate columns
    merged_df = pd.concat([df1, df2], axis=1)

    # Get the directory of the input files
    input_dir = os.path.dirname(file1)

    # Define the output file path
    output_file = os.path.join(input_dir, 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv')

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as: {output_file}")

# Example usage
file1 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-1.csv'  # replace with the path to your first CSV file
file2 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-2.csv'  # replace with the path to your second CSV file

merge_csv_files(file1, file2)

Merged file saved as: xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv
